# NOTICE
Neural Model testing did **NOT** occur for embedded feature type inferencing. Results are copied results from original repository. 

In [1]:
#Copyright 2019 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate,GlobalMaxPooling1D
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU, BatchNormalization
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
np.random.seed(500)

ModuleNotFoundError: No module named 'keras'

In [2]:
# define network parameters
max_features = 128
maxlen = 512

In [3]:
dict_label = {'Numbers': 0, 'List': 1, 'Datetime': 2, 'Sentence': 3,
              'URL': 4, 'Custom Object': 5, 'Unusable': 6}
data = pd.read_csv('data/needs_extraction_data/date_and_time_combined.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:, ['y_act']]
data_LSTM = pd.concat([data['Attribute_name']], axis=1)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,5,10,11,12,13,14,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# data = data.rename(columns={'Num of nans': 'Num_of_nans', 'num of dist_val': 'num_of_dist_val'})

# # data['Num_of_nans'] = [float(data['Num_of_nans'][i])/float(data['Total_val'][i]) for i in data.index]
# # data['num_of_dist_val'] = [float(data['num_of_dist_val'][i])/float(data['Total_val'][i]) for i in data.index]

# data1 = data[['Num_of_nans', 'max_val', 'mean', 'min_val', 'num_of_dist_val','std_dev','castability','extractability', 'len_val','Total_val']]
# data1 = data1.fillna(0)

# data1['Num_of_nans'] = [data1['Num_of_nans'][i]*100/data1['Total_val'][i] for i in data1.index]
# data1['num_of_dist_val'] = [(data1['num_of_dist_val'][i]*1.0)/data1['Total_val'][i] for i in data1.index]

# data1 = data1.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev'})
# data1.loc[data1['scaled_min_val'] > 10000, 'scaled_min_val'] = 10000
# data1.loc[data1['scaled_min_val'] < -10000, 'scaled_min_val'] = -10000
# data1.loc[data1['scaled_max_val'] > 10000, 'scaled_max_val'] = 10000
# data1.loc[data1['scaled_max_val'] < -10000, 'scaled_max_val'] = -10000
# data1.loc[data1['scaled_mean'] > 10000, 'scaled_mean'] = 10000
# data1.loc[data1['scaled_mean'] < -10000, 'scaled_mean'] = -10000
# data1.loc[data1['scaled_std_dev'] > 10000, 'scaled_std_dev'] = 10000
# data1.loc[data1['scaled_std_dev'] < -10000, 'scaled_std_dev'] = -10000
# column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev','num_of_dist_val','Num_of_nans','Total_val']
# x = data1[column_names_to_normalize].values
# x = np.nan_to_num(x)
# x_scaled = StandardScaler().fit_transform(x)
# df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data1.index)
# data1[column_names_to_normalize] = df_temp

# data1.Num_of_nans = data1.Num_of_nans.astype(float)
# data1.num_of_dist_val = data1.num_of_dist_val.astype(float)
# data1.castability = data1.castability.astype(float)
# data1.extractability = data1.extractability.astype(float)
# y.y_act = y.y_act.astype(float)


# data1 = data1[['Num_of_nans', 'scaled_max_val', 'scaled_mean', 'scaled_min_val', 'num_of_dist_val','scaled_std_dev','castability','extractability', 'len_val','Total_val']]


In [5]:
arr = data['Attribute_name'].values
arr = [str(x) for x in arr]

vectorizer = CountVectorizer(ngram_range=(1,1),analyzer='char')
X = vectorizer.fit_transform(arr)
print(len(vectorizer.get_feature_names()))

temp_df = data.drop(['Record_id', 'Attribute_name', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5'],axis=1)
df = pd.DataFrame(X.toarray())
df = pd.concat([df,data], axis=1, sort=False)

1927


In [6]:
key_name = data['Attribute_name']
X_train, X_test, y_train, y_test = train_test_split(
    key_name, y, test_size=0.2, random_state=100)


# atr_train, atr_test = train_test_split(
#     key_name, test_size=0.2, random_state=100)


X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(f"Training set size: {len(X_train)}")

structured_data_train = X_train
structured_data_test = X_test

7400
7400
5122
7400
7400


In [8]:
list_sentences_train = atr_train.values
list_sentences_test = atr_test.values

print(len(list_sentences_train))

# X_train.sample_1 = X_train.sample_1.astype(str)
# X_test.sample_1 = X_test.sample_1.astype(str)

list_sentences_train1 = samp1_train.values
list_sentences_test1 = samp1_test.values

print(list_sentences_train1)

# X_train.sample_2 = X_train.sample_2.astype(str)
# X_test.sample_2 = X_test.sample_2.astype(str)

list_sentences_train2 = samp2_train.values
list_sentences_test2 = samp2_test.values

print(list_sentences_train2)

for i in range(len(list_sentences_train)):
    list_sentences_train[i] = str(list_sentences_train[i])
    list_sentences_train1[i] = str(list_sentences_train1[i])
    list_sentences_train2[i] = str(list_sentences_train2[i])
    
for i in range(len(list_sentences_test)):
    list_sentences_test[i] = str(list_sentences_test[i])
    list_sentences_test1[i] = str(list_sentences_test1[i])
    list_sentences_test2[i] = str(list_sentences_test2[i])    

print(len(list_sentences_train))

7400
['#NULL!' '0' '268' ... 'Hate' '-0.101' '2']
['0' '78' '110' ... nan '0.171' '5']
7400


In [9]:
tokenizer = keras_text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)


tokenizer1 = keras_text.Tokenizer(char_level = True)
tokenizer1.fit_on_texts(list(list_sentences_train1))
# train data
list_tokenized_train1 = tokenizer.texts_to_sequences(list_sentences_train1)
X_t1 = keras_seq.pad_sequences(list_tokenized_train1, maxlen=maxlen)
# test data
list_tokenized_test1 = tokenizer.texts_to_sequences(list_sentences_test1)
X_te1 = keras_seq.pad_sequences(list_tokenized_test1, maxlen=maxlen)


# tokenizer2 = keras_text.Tokenizer(char_level = True)
# tokenizer2.fit_on_texts(list(list_sentences_train1))
# # train data
# list_tokenized_train2 = tokenizer.texts_to_sequences(list_sentences_train2)
# X_t2 = keras_seq.pad_sequences(list_tokenized_train2, maxlen=maxlen)
# # test data
# list_tokenized_test2 = tokenizer.texts_to_sequences(list_sentences_test2)
# X_te2 = keras_seq.pad_sequences(list_tokenized_test2, maxlen=maxlen)

# print(tokenizer)

In [10]:
def build_model(numfilters,embed_size):
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer.word_counts)+1, output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.5)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(numfilters, kernel_size = 3, activation = 'tanh')(x)
        numfilters = numfilters*2
    
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    xy = concatenate(out_conv, axis = -1)  
    print(xy.shape)
    #########################################################
    inp1 = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(inp1)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []

    x = prefilt_x
    for i in range(2):
        x = Conv1D(16*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    out_conv += [GlobalMaxPool1D()(x)]
    x1 = concatenate(out_conv, axis = -1)
    print(x1.shape)
    
    Str_input = Input(shape=(5122,))
    layersfin = keras.layers.concatenate([xy,Str_input])
    print(layersfin.shape)
    x = Dense(500, activation='relu')(layersfin)
    x = Dropout(0.75)(x)
    x = Dense(500, activation='relu')(x)
    x = Dropout(0.75)(x)
    x = Dense(5, activation='softmax')(x)
    model = Model(inputs=[inp,Str_input], outputs=[x])
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

# model = build_model()
# model.summary()

In [11]:
batch_size = 32 # large enough that some other labels come in
epochs = 25

file_path="best_weights.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=20)

callbacks_list = [checkpoint, early] #early
# history = model.fit([X_t,X_t1,structured_data_train], to_categorical(y_train),
#                     validation_data=([X_te,X_te1,structured_data_test], to_categorical(y_test)),
#                     batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)

# # model = build_model()
# # model.summary()


In [12]:
print(X_t)
print(X_t.shape)
# print(y_train[1851:])
print(len(y_train))
y_train = y_train.values
structured_data_train = structured_data_train.values



[[ 0  0  0 ...  2 11  2]
 [ 0  0  0 ... 37 22 39]
 [ 0  0  0 ... 15  5 23]
 ...
 [ 0  0  0 ... 18  5  8]
 [ 0  0  0 ... 28 16 33]
 [ 0  0  0 ...  4  1  2]]
(7400, 512)
7400


In [ ]:
batch_size = 32 # large enough that some other labels come in
epochs = 25

k = 5
kf = KFold(n_splits=k)

# n_filters_grid = [32,64,128,256,512]
# embed_size = [64,128,256,512]

n_filters_grid = [32]
embed_size = [256]
models = []

avgsc_lst,avgsc_val_lst,avgsc_train_lst = [],[],[]
avgsc,avgsc_val,avgsc_train = 0,0,0
i=0
for train_index, test_index in kf.split(X_t):
    file_path= 'best_weights'+str(i)+'.h5'
    checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_acc", mode="max", patience=20)

    callbacks_list = [checkpoint, early] #early
        
    print('\n')
    X_train_cur, X_test_cur = X_t[train_index], X_t[test_index]
    y_train_cur, y_test_cur = y_train[train_index], y_train[test_index]
    structured_data_train_cur, structured_data_test_cur = structured_data_train[train_index],structured_data_train[test_index]
    
    X_train_train, X_val,y_train_train,y_val = train_test_split(X_train_cur,y_train_cur, test_size=0.25,random_state=100)
    structured_data_train_train,structured_data_val = train_test_split(structured_data_train_cur, test_size=0.25,random_state=100)
    
    
    print(X_train_cur)
    
    print(y_train_train.shape)
    print(y_val.shape)
    print(structured_data_train_train.shape)
    print(structured_data_val.shape)
    
    bestPerformingModel = build_model(10,5)
    bestscore = 0
    for ne in n_filters_grid:
        for md in embed_size:
#             clf = RandomForestClassifier(n_estimators=ne,max_depth=md)
            clf = build_model(ne,md)
            history = clf.fit([X_train_cur,structured_data_train_cur], to_categorical(y_train_cur),
                    validation_data=([X_test_cur,structured_data_test_cur], to_categorical(y_test_cur)),
                    batch_size=batch_size, epochs=epochs, shuffle = True, callbacks=callbacks_list)
        
            sc = clf.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
            bestPerformingModel = clf
            if bestscore < sc:
                bestscore = sc
                bestPerformingModel = clf

#     clf.load_weights('best_weights'+str(i)+'.h5')
    print(sc)
    avgsc_val_lst.append(sc[1])
    
    loss, bscr_train = bestPerformingModel.evaluate([X_train_cur,structured_data_train_cur],to_categorical(y_train_cur))
    print(loss, bscr_train)
    loss, bscr_val = bestPerformingModel.evaluate([X_test_cur,structured_data_test_cur],to_categorical(y_test_cur))
    print(loss, bscr_val)    
    loss, bscr = bestPerformingModel.evaluate([X_te,structured_data_test],to_categorical(y_test))
    print(loss, bscr)
    
    models.append(clf)
    
    avgsc = avgsc + bscr
    
    avgsc_train = avgsc_train + bscr_train
    avgsc_val = avgsc_val + bscr_val
    avgsc_lst.append(bscr)
    avgsc_train_lst.append(bscr_train)
    print('The training accuracy is:')
    print(bscr_train)
    print('The validation accuracy is:')
    print(bscr_val)    
    print('The test accuracy is:')    
    print(bscr)
    print('\n')
    i=i+1



[[ 0  0  0 ...  3 11 10]
 [ 0  0  0 ... 22  6 33]
 [ 0  0  0 ...  4  1  2]
 ...
 [ 0  0  0 ... 18  5  8]
 [ 0  0  0 ... 28 16 33]
 [ 0  0  0 ...  4  1  2]]
(4440, 1)
(1480, 1)
(4440, 5122)
(1480, 5122)
(?, 40)
(?, 64)
(?, 5162)
(?, 128)
(?, 64)
(?, 5250)
Train on 5920 samples, validate on 1480 samples
Epoch 1/25


In [23]:
from pandas import DataFrame
print(models)
print(avgsc_train_lst)
print(avgsc_val_lst)
print(avgsc_lst)

# i = avgsc_lst.index(max(avgsc_lst))
# bestPerformingModel = models[i]

print(avgsc_train/k)
print(avgsc_val/k)
print(avgsc/k)


loss, bscr = bestPerformingModel.evaluate([X_te,structured_data_test],to_categorical(y_test))
y_pred = bestPerformingModel.predict([X_te,structured_data_test])

df = DataFrame.from_records(y_pred)
# print(df)
df.to_csv('cnn_predictions.csv',index=False)

print(y_pred)
y_pred = [np.argmax(i) for i in y_pred]
cm = confusion_matrix(y_test,y_pred)
print(cm)


[<keras.engine.training.Model object at 0x7f99d0804d90>, <keras.engine.training.Model object at 0x7f99d4cb1ed0>, <keras.engine.training.Model object at 0x7f99d1638610>, <keras.engine.training.Model object at 0x7f99d03ed990>, <keras.engine.training.Model object at 0x7f99b0698cd0>]
[0.9905405405405405, 0.9912162162162163, 0.9908783783783783, 0.9896959459459459, 0.9880067567567568]
[0.8770270270270271, 0.8675675675675676, 0.8675675675675676, 0.8797297297297297, 0.8736486486486487]
[0.8816855757832849, 0.8833063213262347, 0.8827660728119181, 0.8811453272689683, 0.8827660721678917]
0.9900675675675675
0.8731081081081081
0.8823338738716595
1851/1851 [==============================] - 0s 76us/step
[[1.00000000e+00 2.97056975e-13 1.15593654e-11 1.55729271e-12
  6.76322687e-09]
 [1.00000000e+00 1.13202946e-15 1.22582120e-12 1.07208461e-11
  4.43889364e-13]
 [3.68195265e-06 8.87664100e-06 5.86224178e-06 3.94786093e-06
  9.99977589e-01]
 ...
 [4.21263212e-06 7.87665613e-07 9.99491215e-01 5.0122541

In [219]:
y_pred_df = pd.DataFrame({'y_pred':y_pred})
# print(y_pred)
dfn = pd.concat([y_pred_df,y_test],axis=1)
dfn.to_csv('dfny.csv')

In [15]:
bestone = load_model('best_weights1.h5')
print('---')
loss, acc = bestone.evaluate([X_t,structured_data_train],to_categorical(y_train),verbose=1)
print(loss, acc)
loss, acc = bestone.evaluate([X_te,structured_data_test],to_categorical(y_test),verbose=1)
print(loss, acc)

---
7400/7400 [==============================] - 5s 678us/step
(0.1644142657493216, 0.9675675675675676)
1851/1851 [==============================] - 0s 123us/step
(0.5964181145772878, 0.8849270665471712)


In [38]:
data1_train, data1_test = train_test_split(data1, test_size=0.2,random_state=100)
data1_test = data1_test.reset_index(drop=True)
atr_test.reset_index(inplace=True,drop=True)

y_pred = bestone.predict([X_te,structured_data_test])
i=0
for x in y_pred:
    j=0
    for y in x:
        y_pred[i][j] = round(y_pred[i][j],3)
        j=j+1
    i=i+1
# print(y_pred)
fone =  pd.DataFrame(y_pred)

y_pred = [np.argmax(i) for i in y_pred]
# print(y_pred)

sone =  pd.DataFrame({'y_pred':y_pred})

data = pd.concat([atr_test,samp1_test,data1_test,fone,sone,y_test], axis=1)
data.to_csv('cnn_pred.csv')
